In [3]:
import pandas as pd
import numpy as np

# Load the dataframes
mod_df = pd.read_excel("mod_df_gb.xlsx")
ad_df = pd.read_excel("ad_df_gb.xlsx")

# Sort both dataframes by their respective scores in descending order
sorted_mod_df = mod_df.sort_values(by="mod_score_pred", ascending=False)
sorted_ad_df = ad_df.sort_values(by="ad_score_pred", ascending=False)

# Calculate the number of ads each moderator should get
num_ads_per_mod = len(sorted_ad_df) // len(sorted_mod_df)

# Initialize a dictionary to track the number of ads assigned to each moderator
assigned_ads_count = {moderator: 0 for moderator in sorted_mod_df['moderator']}

# Initialize a list to store the matches
matches = []

# Iterate through the ads and assign them to moderators
for _, ad_row in sorted_ad_df.iterrows():
    country = ad_row['delivery_country']
    suitable_mods = sorted_mod_df[sorted_mod_df[f'"{country}"'] == 1.0]
    
    # Filter out rows with NaN values in the 'moderator' column
    suitable_mods = suitable_mods.dropna(subset=['moderator'])
    
    if not suitable_mods.empty:
        # Find a moderator with the least number of assigned ads
        selected_mod = None
        min_assigned_ads = float('inf')
        for _, mod_row in suitable_mods.iterrows():
            mod_id = mod_row['moderator']
            if assigned_ads_count[mod_id] < min_assigned_ads:
                selected_mod = mod_id
                min_assigned_ads = assigned_ads_count[mod_id]
        
        # Update the assigned_ads_count for the selected moderator
        assigned_ads_count[selected_mod] += 1
        
        # Append the match to the matches list
        matches.append((ad_row['ad_id'], selected_mod))

# Create a DataFrame from the matches
matched_df = pd.DataFrame(matches, columns=['ad_id', 'moderator'])

# Save the matched pairs to a file
matched_df.to_excel("final_matched_pairs.xlsx", index=False)

